In [23]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [26]:
train_dataset = KeypointsDataset("data/images","data/data_train.json")
val_dataset = KeypointsDataset("data/images","data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [27]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

In [28]:
model = model.to(device)

In [29]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [30]:
epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14837.728515625
Epoch 0, iter 10, loss: 14096.1943359375
Epoch 0, iter 20, loss: 13638.7080078125
Epoch 0, iter 30, loss: 14119.1845703125
Epoch 0, iter 40, loss: 13009.7197265625
Epoch 0, iter 50, loss: 13661.501953125
Epoch 0, iter 60, loss: 12744.421875
Epoch 0, iter 70, loss: 13384.953125
Epoch 0, iter 80, loss: 12214.1953125
Epoch 0, iter 90, loss: 11760.2431640625
Epoch 0, iter 100, loss: 10466.015625
Epoch 0, iter 110, loss: 10950.197265625
Epoch 0, iter 120, loss: 9942.451171875
Epoch 0, iter 130, loss: 10207.671875
Epoch 0, iter 140, loss: 9315.3369140625
Epoch 0, iter 150, loss: 8950.5712890625
Epoch 0, iter 160, loss: 9749.6357421875
Epoch 0, iter 170, loss: 8652.8740234375
Epoch 0, iter 180, loss: 10052.380859375
Epoch 0, iter 190, loss: 8371.939453125
Epoch 0, iter 200, loss: 7385.29443359375
Epoch 0, iter 210, loss: 8604.021484375
Epoch 0, iter 220, loss: 7189.66162109375
Epoch 0, iter 230, loss: 6908.91650390625
Epoch 0, iter 240, loss: 7048.382812

In [42]:
torch.save(model.state_dict(), "keypoints_model.pth")